# Proyecto Final - Ensemble Learning (Aprendizaje por ensamblado)
## Parte 1 - Entrenamieto, selección y validación.

**Curso:** Statistical Learning

**Catedrático:** Ing. Luis Leal

**Estudiante:** Dany Rafael Díaz Lux (21000864)

**Objetivo:** Hacer clasificación binaria para determinar si una persona sobrevive o no al hundimiento del Titanic.

## Cargar información

In [122]:
# Import required libraries
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import datetime as dt
import matplotlib.pylab as plt
import numpy as np 
import os.path
import pandas as pd
import sklearn.metrics as mts
import tensorflow as tf
print('Tensor flow version: ' + tf.__version__)

Tensor flow version: 2.4.1


In [2]:
df = pd.read_csv("data_titanic_proyecto.csv")
display(df)

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,S,Middle,M,N
887,888,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,S,Upper,F,Y
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,NaN,S,Lower,F,N
889,890,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,C,Upper,M,Y


## Pre-procesamiento

### Ignorar columnas identificadoras

In [3]:
# Omitir columnas "PassengerId", "Name", "Ticket" y "Cabin" por ser columnas que tratan de distinguir a cada individuo y
# no buscan indicar una característica general.
caracteristicas = df.iloc[:,[2,3,4,6,8,9,10]]
etiquetas = df.iloc[:,11]
display(caracteristicas.head())
display(etiquetas.head())

,Age,SibSp,Parch,Fare,Embarked,passenger_class,passenger_sex
0,22.0,1,0,7.2500,S,Lower,M
1,38.0,1,0,71.2833,C,Upper,F
2,26.0,0,0,7.9250,S,Lower,F
3,35.0,1,0,53.1000,S,Upper,F
4,35.0,0,0,8.0500,S,Lower,M


0    N
1    Y
2    Y
3    Y
4    N
Name: passenger_survived, dtype: object

### Detectar columnas con información faltante (NaN), determinar porcentaje, e imputación.

In [4]:
# Listar columnas con valores NaN
columnasConNaN = caracteristicas.columns[caracteristicas.isna().any()].tolist()
for columna in columnasConNaN:
    print('Porcentaje NaN en ', columna, ':', round(100 * \
            len(caracteristicas[caracteristicas[columna].isna()]) / len(caracteristicas), 2), '%')

Porcentaje NaN en  Age : 19.87 %
Porcentaje NaN en  Embarked : 0.22 %


In [5]:
# Se imputará la información faltante con la media de los datos en la columna "Age"
caracteristicas['Age'].fillna(value=caracteristicas['Age'].mean(), inplace=True)

C:\Users\Dany\anaconda3\envs\py_galileo_2021\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [6]:
# Se imputará la información faltante con la moda de los datos en la columna "Embarked" por ser categórica
caracteristicas['Embarked'].fillna(value=caracteristicas['Embarked'].mode()[0], inplace=True)

### One hot encoding en columnas categóricas

In [18]:
# Realizar one hot encoding en columnas: "Enbarked", "passenger_class", "passenger_sex" y "passenger_survived"
caracteristicasConOhe = caracteristicas.join(pd.get_dummies(caracteristicas.Embarked, prefix='Embarked'))
caracteristicasConOhe = caracteristicasConOhe.join(pd.get_dummies(caracteristicasConOhe.passenger_class, prefix='passenger_class'))
caracteristicasConOhe = caracteristicasConOhe.join(\
                                pd.get_dummies(caracteristicasConOhe.passenger_sex, prefix='passenger_sex', drop_first=True))
caracteristicasConOhe = caracteristicasConOhe.loc[:, ~caracteristicasConOhe.columns.isin(['Embarked', 'passenger_class', 'passenger_sex'])]
etiquetasConOhe = pd.get_dummies(etiquetas, prefix='passenger_survived', drop_first=True)
display(caracteristicasConOhe.head())
display(etiquetasConOhe.head())

,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper,passenger_sex_M
0,22.0,1,0,7.2500,0,0,1,1,0,0,1
1,38.0,1,0,71.2833,1,0,0,0,0,1,0
2,26.0,0,0,7.9250,0,0,1,1,0,0,0
3,35.0,1,0,53.1000,0,0,1,0,0,1,0
4,35.0,0,0,8.0500,0,0,1,1,0,0,1


,passenger_survived_Y
0,0
1,1
2,1
3,1
4,0


## División entre datos de entrenamiento y datos de validación

In [32]:
# Primera división 80-20 entre el set de entrenamiento completo y datos de validación final.
caracteristicasConOhe_completeTrain, caracteristicasConOhe_finalTest, \
    etiquetasConOhe_completeTrain, etiquetasConOhe_finalTest = \
    train_test_split(caracteristicasConOhe, etiquetasConOhe, test_size=0.2, random_state=2022)

# Segunda división 80-20 entre set de entrenamiento y set de pruebas
caracteristicasConOhe_train, caracteristicasConOhe_test, \
    etiquetasConOhe_train, etiquetasConOhe_test = \
    train_test_split(caracteristicasConOhe_completeTrain, etiquetasConOhe_completeTrain, test_size=0.2, random_state=2022)

print('Número de datos de entrenamiento:', len(caracteristicasConOhe_train))
print('Número de datos de pruebas:', len(caracteristicasConOhe_test))
print('Número de datos de validación final:', len(caracteristicasConOhe_finalTest))

Número de datos de entrenamiento: 569
Número de datos de pruebas: 143
Número de datos de validación final: 179


### Escalar características numéricas


In [33]:
# Se aplicará estandarización de datos como método de estandarización
def estandarizar(x, mediaEntrenamiento = None, desviacionEntrenamiento = None):
    if(mediaEntrenamiento == None or desviacionEntrenamiento == None):
        # Si no se reciben datos de entrenamiento, se calculan de los features "x" recibidos
        media = np.mean(x)
        desviacion = np.std(x)
    else:
        media = mediaEntrenamiento
        desviacion = desviacionEntrenamiento
    
    return (x - media) / (desviacion), media, desviacion

In [95]:
# Aplicar estandarización a columnas "Age", "SibSp", "Parch" y "Fare"
caracteristicasConOheEstandarizadas = caracteristicasConOhe_train.join(pd.DataFrame())
caracteristicasConOheEstandarizadas['Age'], mediaAge_train, desviacionAge_train = \
            estandarizar(caracteristicasConOheEstandarizadas['Age'])
caracteristicasConOheEstandarizadas['SibSp'], mediaSibSp_train, desviacionSibSp_train = \
            estandarizar(caracteristicasConOheEstandarizadas['SibSp'])
caracteristicasConOheEstandarizadas['Parch'], mediaParch_train, desviacionParch_train = \
            estandarizar(caracteristicasConOheEstandarizadas['Parch'])
caracteristicasConOheEstandarizadas['Fare'], mediaFare_train, desviacionFare_train = \
            estandarizar(caracteristicasConOheEstandarizadas['Fare'])

# Aplicar estandarización a datos de prueba
caracteristicasConOheEstandarizadas_test = caracteristicasConOhe_test.join(pd.DataFrame())
caracteristicasConOheEstandarizadas_test['Age'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas_test['Age'], mediaAge_train, desviacionAge_train)
caracteristicasConOheEstandarizadas_test['SibSp'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas_test['SibSp'], mediaSibSp_train, desviacionSibSp_train)
caracteristicasConOheEstandarizadas_test['Parch'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas_test['Parch'], mediaParch_train, desviacionParch_train)
caracteristicasConOheEstandarizadas_test['Fare'], _, _ = \
            estandarizar(caracteristicasConOheEstandarizadas_test['Fare'], mediaFare_train, desviacionFare_train)

display(caracteristicasConOheEstandarizadas.head())
display(caracteristicasConOheEstandarizadas_test.head())

,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper,passenger_sex_M
357,0.689198,-0.474164,-0.485553,-0.392896,0,0,1,0,1,0,0
613,0.030567,-0.474164,-0.485553,-0.496634,0,1,0,1,0,0,1
868,0.030567,-0.474164,-0.485553,-0.462055,0,0,1,1,0,0,1
414,1.165266,-0.474164,-0.485553,-0.493176,0,0,1,1,0,0,1
863,0.030567,6.443769,2.003451,0.724512,0,0,1,1,0,0,0


,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,passenger_class_Lower,passenger_class_Middle,passenger_class_Upper,passenger_sex_M
75,-0.342283,-0.474164,-0.485553,-0.498610,0,0,1,1,0,0,1
620,-0.183593,0.390578,-0.485553,-0.364161,1,0,0,1,0,0,1
562,-0.104249,-0.474164,-0.485553,-0.383016,0,0,1,0,1,0,1
129,1.244611,-0.474164,-0.485553,-0.511948,0,0,1,1,0,0,1
631,1.720679,-0.474164,-0.485553,-0.510383,0,0,1,1,0,0,1


## Árbol de decisión

In [189]:
# Función para crear un nuevo dataframe vacío con las columnas esperadas en bitácora
def nuevoDataframeParaBitacora():
    return pd.DataFrame({'tipomodelo': [], 'fecha': [], 'configuracion': [], 'error_train': [],\
                        'accuracy_train': [], 'precision_train': [], 'recall_train': [], 'f1_score_train': [],\
                        'accuracy_test': [], 'precision_test': [], 'recall_test': [], 'f1_score_test': []})

In [217]:
nombreBitacora = 'bitacora_modelos.csv'
# Función que determinará si una configuración de un experimento en particular ya se encuentra en la bitácora
def existeConfiguracion(configuracion):
    # Si existe bitácora, buscar si la configuración ya existe
    if os.path.exists(nombreBitacora):
        dfBitacora = pd.read_csv(nombreBitacora)
        if(not dfBitacora['configuracion'].where(dfBitacora['configuracion'] == configuracion).isna().all()):
            return True
    return False

# Función que agregará una configuración de un experimento a la bitácora con sus métricas más importantes
def manejarBitacora(tipoModelo, configuracion, y_train, y_pred_train, y_test, y_pred_test):
    dfNewConfiguracion = nuevoDataframeParaBitacora()
    dataConfiguracion = [tipoModelo, dt.datetime.now(), configuracion, '']
    dataConfiguracion.append(mts.accuracy_score(y_train, y_pred_train))
    dataConfiguracion.append(mts.precision_score(y_train, y_pred_train))
    dataConfiguracion.append(mts.recall_score(y_train, y_pred_train))
    dataConfiguracion.append(mts.f1_score(y_train, y_pred_train))
    dataConfiguracion.append(mts.accuracy_score(y_test, y_pred_test))
    dataConfiguracion.append(mts.precision_score(y_test, y_pred_test))
    dataConfiguracion.append(mts.recall_score(y_test, y_pred_test))
    dataConfiguracion.append(mts.f1_score(y_test, y_pred_test))
    dfNewConfiguracion.loc[len(dfNewConfiguracion)] = dataConfiguracion

    # Agregar configuración en bitácora
    if os.path.exists(nombreBitacora):
        dfNewConfiguracion.to_csv(nombreBitacora, mode='a', index=False, header=False)
    else:
        dfNewConfiguracion.to_csv(nombreBitacora, index=False)

In [218]:
tipoArbolDecision = 'ArbolDecision'
# Función para la creación de modelo de árbol de decisión y registro de métricas en bitácora (en caso de no existir en la bitácora)
def crearEvaluarArbolDecision(X, y, X_test, y_test, criterio='gini', profundidadMaxima=None):
    # Crear modelo con parámetros enviados
    modelo = DecisionTreeClassifier(criterion=criterio, max_depth=profundidadMaxima, random_state=2022)
    modelo.fit(X, y)
    # Crear cadena de configuración de árbol de decisión
    configuracion = tipoArbolDecision + '_criterio=' + criterio + '_profundidadMaxima=' + str(profundidadMaxima)
    configuracionYaExiste = existeConfiguracion(configuracion)
        
    # Si no existe configuración en bitácora, agregar configuración a bitácora y métricas    
    if(not configuracionYaExiste):
        # Calcular errores de predicción para entrenamiento y test
        y_pred_train = modelo.predict(X)
        y_pred_test = modelo.predict(X_test)
        manejarBitacora(tipoArbolDecision, configuracion, y, y_pred_train, y_test, y_pred_test)

    return modelo

### Creación de diferentes árboles de decisión y análisis de métricas en bitácora

In [222]:
# Hiper-parámetros a experimentar
criterios = ['gini', 'entropy']
maxProfundidades = [None, 10, 50, 100]

for criterio in criterios:
    for maxProfundidad in maxProfundidades:
        crearEvaluarArbolDecision(caracteristicasConOheEstandarizadas, etiquetasConOhe_train, \
                          caracteristicasConOheEstandarizadas_test, etiquetasConOhe_test, criterio, maxProfundidad)

In [240]:
def mostrarResultadosBitacora(tipoModelo):
    dfBitacora = pd.read_csv(nombreBitacora)
    dfBitacora.where(dfBitacora['tipomodelo'] == tipoModelo, inplace = True)
    pd.set_option("display.max_colwidth", 1000)
    display(dfBitacora[['configuracion', 'accuracy_train', 'accuracy_test', 'f1_score_train', 'f1_score_test']]\
            .sort_values(by='accuracy_test', ascending=False).head())

La bitácora señala que los mejores resultados en __accuracy__ para los datos de prueba se obtuvieron con el criterio 'entropy' y con menor profundidad. Cómo __accuracy__ en los datos de pruebas aún es menor al solicitado (80%) se probarán más hiperparámetros hasta alcanzar al menos un __accuracy__ de 80%.

In [242]:
# Hiper-parámetros a experimentar
criterios = ['gini', 'entropy']
maxProfundidades = [3, 5, 9]

for criterio in criterios:
    for maxProfundidad in maxProfundidades:
        crearEvaluarArbolDecision(caracteristicasConOheEstandarizadas, etiquetasConOhe_train, \
                          caracteristicasConOheEstandarizadas_test, etiquetasConOhe_test, criterio, maxProfundidad)
        
mostrarResultadosBitacora(tipoArbolDecision)

,configuracion,accuracy_train,accuracy_test,f1_score_train,f1_score_test
9,ArbolDecision_criterio=gini_profundidadMaxima=5,0.861160,0.825175,0.791557,0.778761
11,ArbolDecision_criterio=entropy_profundidadMaxima=3,0.833040,0.818182,0.767726,0.790323
8,ArbolDecision_criterio=gini_profundidadMaxima=3,0.833040,0.804196,0.766585,0.770492
12,ArbolDecision_criterio=entropy_profundidadMaxima=5,0.850615,0.797203,0.789082,0.760331
13,ArbolDecision_criterio=entropy_profundidadMaxima=9,0.894552,0.797203,0.839572,0.747826


Los nuevos resultados señalan que una menor profundidad claramente mejora la exactitud. El criterio (gini o entropy) no parece afectar claramente en la mejora de exactitud. Se realizarán más pruebas con la profundidad

In [243]:
# Hiper-parámetros a experimentar
criterios = ['gini', 'entropy']
maxProfundidades = [2, 4, 6, 7, 8]

for criterio in criterios:
    for maxProfundidad in maxProfundidades:
        crearEvaluarArbolDecision(caracteristicasConOheEstandarizadas, etiquetasConOhe_train, \
                          caracteristicasConOheEstandarizadas_test, etiquetasConOhe_test, criterio, maxProfundidad)
        
mostrarResultadosBitacora(tipoArbolDecision)

,configuracion,accuracy_train,accuracy_test,f1_score_train,f1_score_test
15,ArbolDecision_criterio=gini_profundidadMaxima=4,0.847100,0.832168,0.789346,0.809524
9,ArbolDecision_criterio=gini_profundidadMaxima=5,0.861160,0.825175,0.791557,0.778761
21,ArbolDecision_criterio=entropy_profundidadMaxima=6,0.869947,0.818182,0.802139,0.771930
20,ArbolDecision_criterio=entropy_profundidadMaxima=4,0.841828,0.818182,0.776119,0.790323
11,ArbolDecision_criterio=entropy_profundidadMaxima=3,0.833040,0.818182,0.767726,0.790323


Después de las pruebas realizadas, se elige un árbol de decisión con criterio __gini__ y profundidad máxima de __4__, pues reporta las mejores métricas de exactitud para los datos de prueba y una exactitud aceptable en los datos de entrenamiento.

In [244]:
modeloArbolDecision = crearEvaluarArbolDecision(caracteristicasConOheEstandarizadas, etiquetasConOhe_train, \
                          caracteristicasConOheEstandarizadas_test, etiquetasConOhe_test, 'gini', 4)

## Support Vector Machine (SVM)

## Naive Bayes

## Regresión Logística

## Investigación K-fold

## Tabla de predicciones

## Tabla de métricas de evaluación

## Conclusiones